<a href="https://colab.research.google.com/github/boyoung21/ESAA/blob/main/0926_%EC%88%98%EC%83%81%EC%9E%91_%EB%A6%AC%EB%B7%B0_%EC%B2%AD%EA%B2%BD%EC%B1%84_%EC%84%B1%EC%9E%A5_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 청경채 성장 예측 AI 경진대회

https://dacon.io/competitions/official/235961/codeshare/6591?page=1&dtype=recent

대회 목표: 농업 분야에도 AI 기술이 널리 사용되는 현재, IT 기술을 동원한 스마트팜 등 더욱 효율적인 작물 재배가 가능해지고 있음. 이에 환경 데이터로부터 청경채의 일별 잎 면적 증감률을 예측하는 AI 알고리즘을 개발하고자 함.

# 코드 분석

분석 목표 설정: 배추과 식물의 생장에 영향을 미치는 요소를 파악함.
- 온도처리에 따른 생리 반응 변화나 LED 광원이 미치는 영향 등
> 청경채의 생장에 큰 영향을 주는 요소를 바탕으로 잎의 생장을 예측하고자

이를 통해 남은 시간을 날짜 단위로 수정 후, 장비들이 켜져 있는 시간으로 그룹화. 또, LED 조명이 모두 꺼져 있는 시간을 추가. 그리고 파장별 추정광량의 max, min 값 추가, 이들의 차이값도 추가

온도 습다가 미치는 영향을 고려하여 내부 온도 관측치의 max와 min값 추가, 이들의 차이인 일교차 추가, 습도와 온도를 곱한 값 추가, 습도온도co2를 모두 곱한 값 추가

생장 주기 반영을 위해 첫째날부터 마지막날까지를 센 growth_day 칼럼 추가.

분석 방법
1. 전처리를 통해 컬럼과 데이터 수정
- 다양한 변인들이 식물의 생장에 직접적인 영향을 미치기까지는 시간이 걸릴 것으로 가정함
- 미리 제공돼 있는 생장률을 예측에 사용하는 것이 더 적절하다고 판단함
- 해당 시점보다 하루 앞선 시점, T+1의 생장률로 데이터세트를 형성 -> T에서의 생장률은 드랍함.

구체적인 전처리: 케이스별로 따로 예측할 경우 새로운 데이터에 유연하게 대응하지 못할 것이라 판단하여 케이스들을 모두 취합한 train, test 셋 생성

결측치 보간: 선형보간법 사용

- 선형보간법: 끝점의 값이 주어졌을 때 그 사이에 위치한 값을 추정하기 위하여 직선거리에 따라 선형적으로 계산하는 방법

- 내부와는 관계없는 '외부'가 포함된 컬럼 삭제, 카메라 상태 컬럼 삭제, 이 외 비슷한 컬럼명들 이름 변경

온도 관련 컬럼, 파장 관련 컬럼, 변수 컬럼 추가(파생변수들)을 세세하게 분류함

2. 청경채의 생장과 관련이 있을 것으로 예상되는 변수 컬럼을 추가함

- LED, 온도, 날짜 등을 활용한 파생변수들. 1번에 자세히 설명

3. 교차 검증을 통해 예측

- 먼저 Standard Scaler로 피처들의 스케일을 맞춰 줌

- XGBoost를 사용

- 5-fold로 교차검증

4. 성능이 좋은 변수 컬럼을 추가

장비들이 작동하고 있는 시간만 추출하는 함수, 컬럼 추가/삭제 함수 등의 사용자 정의 함수를 직접 지정하여 코드의 효율성을 높임

모델링 자체는 간단하나, 실제 배추과 식물들의 생장 환경을 조사하여 외부 지식을 바탕으로 파생변수를 생성한 점, 그리고 외부지식에 부합하지 않는 컬럼들을 선별하여 삭제하고 피처를 간단화한 것이 인상 깊음. 파생변수를 어떤 기준에 따라 생성하면 좋을지에 대한 좋은 기준이 됨.
